# Age & Gender Classifier using Deep CNNs  - 

**Dataset : Adience**

17452 records with attributes age and gender

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Essential libraries
import numpy as np 
import pandas as pd
import os
import glob
import pandas as pd
import seaborn as sns
import scipy.io
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn import preprocessing
from collections import Counter
import pickle

import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import SeparableConv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers import SpatialDropout2D
from keras.layers.core import Lambda
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.regularizers import l2



from keras import applications,activations
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras import optimizers,utils
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D,BatchNormalization,ZeroPadding2D, Input
from keras.layers import Conv2D, Activation,MaxPooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.utils.vis_utils import plot_model

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sb
plt.style.use('classic')
%matplotlib inline
mpl.rcParams['figure.figsize'] = [10,5]


In [ ]:

TRAIN_TEST_SPLIT = 0.8
IM_WIDTH = IM_HEIGHT = 198

dataset_dict = {
    'gender_id': {
        0: 'male',
        1: 'female'
    }
}

dataset_dict['gender_alias'] = dict((g, i) for i, g in dataset_dict['gender_id'].items())    # (Gender: id)


In [ ]:
path_list = []
age_list = []
gender_list = []

def genderParser(line,index):
    if(index == np.nan or index == 'int'):
        if(line.split()[4] == 'm' or line.split()[4] == 'f'):
            return line.split()[4]
    if(index == 'tuple'):
        if(line.split()[5] == 'm' or line.split()[5] == 'f'):
            return line.split()[5]
    return np.nan

def Parser(line):
    if(line.split()[3] == 'None'):
        return np.nan,genderParser(line,np.nan)
    
    if(line.split()[3][0] == '('):
        return line.split()[3] + line.split()[4],genderParser(line,'tuple')

    else:
        return line.split()[3],genderParser(line,'int')

def age_lambda_parser(age,strategy='mean'):
    if (strategy == 'mean'):
        return int(np.mean(list(literal_eval(age))))
    elif (strategy == 'first'):
        return int(literal_eval(age)[0])
    if (strategy == 'last'):
        return int(literal_eval(age)[1])
    


def parse_dataset(DATA_FILE,IMAGE_FOLDER_NAME,IMAGE_PREFIX):
    age_list=[]
    gender_list=[]
    path_list=[]
    for fold in range(5):
        with open(DATA_FILE.format(fold)) as file:
            index=0
            next(file)
            for line in file:
                line = line.rstrip()
                path = IMAGE_FOLDER_NAME + str(line.split()[0]) + "/"+ IMAGE_PREFIX.format(str(line.split()[2]),str(line.split()[1]))

                

                age,gender = Parser(line)

                path_list.append(path)
                age_list.append(age)
                gender_list.append(gender)
    return age_list,gender_list,path_list


def processing(DATA_FILE,IMAGE_FOLDER_NAME,IMAGE_PREFIX,bin_width=10):
    
    age_list,gender_list,path_list = parse_dataset(DATA_FILE,IMAGE_FOLDER_NAME,IMAGE_PREFIX)
    
    
    df = pd.DataFrame()
    df['age'] = age_list
    df['gender'] = gender_list
    df['file'] = path_list
    
    
    df=df.dropna()
    df['age'] = df['age'].apply(lambda x: int(x) if x[0] != '(' else age_lambda_parser(x,strategy='mean'))
    bins = np.arange(0,61, bin_width) #max age = 100 min age = 0 
    df['age_range']  = pd.cut(df['age'] , bins,include_lowest=True)
    df['age_range'] = df['age_range'].astype(str)
    df['age_range']=df['age_range'].replace('nan',str(pd.Interval(left=60.0, right=100.0)))
    
    
    le = preprocessing.LabelEncoder()
    df['encoded_age'] = le.fit_transform(df['age_range'])
    
    df['gender'] = df['gender'].apply(lambda x: 'male' if x == 'm' else 'female')
    
    return df



def saveModelHistory(model_history,iteration,path):
    with open(path.format(iteration), 'wb') as fp:
        pickle.dump(model_history, fp)
        
def loadModelHistory(iteration,path):
    with open (path.format(iteration), 'rb') as fp:
        model_history = pickle.load(fp)  
    return model_history

In [ ]:
ADIENCE_PATHS = {
    'DB_PATH':'./adiencedb',
    'DATA_FILE':'./adiencedb/fold_{}_data.txt',
    'IMAGE_FOLDER_NAME':'./adiencedb/faces/',
    'IMAGE_PREFIX':'coarse_tilt_aligned_face.{}.{}',
    'MODEL_SAVED':'./model_outputs_adience/iteration_{}',
    'MODEL_HISTORY':'./model_outputs_adience/model_history_{}'
}

In [ ]:
df = processing(ADIENCE_PATHS['DATA_FILE'],ADIENCE_PATHS['IMAGE_FOLDER_NAME'],ADIENCE_PATHS['IMAGE_PREFIX'],bin_width=10)

In [ ]:
df.head()

# Settings

To change number of encoded variables in AGE :

    1. change AGE_CAT_NUM in generate images
    
    2. change output dim in age model
    
    3. change class weights if applicable

In [ ]:
count = Counter(df['encoded_age'])
age_df = pd.DataFrame.from_dict(count, orient='index')
age_df.sort_index().plot(kind='bar')
plt.show()


## Data Generator
In order to input our data to our Keras multi-output model, we have a helper object to work as a data generator for our dataset. This will be done by generating batches of data, which will be used to feed our multi-output model with both the images and their labels (instead of just loading all the dataset into the memory at once, which might lead to an out of memory error).

In [ ]:
from tensorflow.keras.utils import to_categorical
from PIL import Image
p = np.random.permutation(len(df))

train_up_to = int(len(df) * TRAIN_TEST_SPLIT)      
train_idx = p[:train_up_to]
val_idx = p[train_up_to:]

# converts alias to id
df['gender_id'] = df['gender'].map(lambda gender: dataset_dict['gender_alias'][gender])


            
def preprocess_image(img_path):   # Used to perform some minor preprocessing on the image before inputting into the network.
    
    im = Image.open(img_path)
    im = im.resize((IM_WIDTH, IM_HEIGHT))
    im = np.array(im) / 255.0
    
    return im
        
def generate_images(image_idx, is_training, batch_size=16,AGE_CAT_NUM=7):  # Used to generate a batch with images when training/validating our model.
    
    # arrays to store our batched data
    images, ages, races, genders = [], [], [], []

    while True:
        for idx in image_idx:
            person = df.iloc[idx]
            
            age = person['encoded_age']
            # race = person['race_id']
            gender = person['gender_id']
            file = person['file']
            
            im = preprocess_image(file)
            
            # races.append(to_categorical(race, len(dataset_dict['race_id'])))
            genders.append(to_categorical(gender, len(dataset_dict['gender_id'])))
            ages.append(to_categorical(age,AGE_CAT_NUM))
            images.append(im)
            
            # yielding condition
            if len(images) >= batch_size:
                yield np.array(images), [np.array(ages), np.array(genders)]
                images, ages, genders = [], [], []
                
        if not is_training:
            break
            

## Model Architecture

In [ ]:
def make_default_hidden_layers(inputs):

    x = SeparableConv2D(32, (3, 3), padding="same")(inputs)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(3, 3))(x)
    x = BatchNormalization(axis=-1)(x)

    x = SeparableConv2D(64, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = BatchNormalization(axis=-1)(x)

    x = SeparableConv2D(128, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = BatchNormalization(axis=-1)(x)

    x = SeparableConv2D(128, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = SpatialDropout2D(0.1)(x)
    x = BatchNormalization(axis=-1)(x)

    x = SeparableConv2D(256, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = SpatialDropout2D(0.1)(x)
    x = BatchNormalization(axis=-1)(x)

    x = SeparableConv2D(256, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = SpatialDropout2D(0.15)(x)
    x = BatchNormalization(axis=-1)(x)

    x = SeparableConv2D(256, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = SpatialDropout2D(0.15)(x)
    x = BatchNormalization(axis=-1)(x)

    return x

def build_gender_branch(inputs):
  
    x = make_default_hidden_layers(inputs)

    x = Flatten()(x)
    x = Dense(64)(x)
    x = Activation("relu")(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)

    x = Dense(32)(x)
    x = Activation("relu")(x)
    x = Dropout(0.25)(x)
    x = BatchNormalization()(x)
  
    x = Dense(2)(x)
    x = Activation("softmax", name="gender_output")(x)

    return x

def build_age_branch(inputs):   

    x = make_default_hidden_layers(inputs)
    
    x = Flatten()(x)
    x = Dense(128, kernel_regularizer=l2(0.03))(x)
    x = Activation("relu")(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)

    x = Dense(64)(x)
    x = Activation("relu")(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)
    
    x = Dense(32)(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)

    x = Dense(7)(x)
    x = Activation("softmax", name="age_output")(x)  

    return x


def assemble_model(width, height):
  
    input_shape = (height, width, 3)
    inputs = Input(shape=input_shape)

    age_branch = build_age_branch(inputs)
    gender_branch = build_gender_branch(inputs)

    model = Model(inputs=inputs, outputs = [age_branch, gender_branch], name="face_net")

    return model
    
model = assemble_model(198, 198)

In [ ]:
model.summary()

# Plot model only works on colab

In [ ]:
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True) 

In [ ]:
from sklearn.utils import class_weight
weights = class_weight.compute_class_weight('balanced' ,np.array(list(range(7))) ,np.array(df['encoded_age']))

class_weights = dict(zip(list(range(7)), weights))
print(class_weights)

In [ ]:
import math
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LearningRateScheduler

def step_decay(epoch):
	initial_lrate = 0.002
	drop = 0.5
	epochs_drop = 7.0
	lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lrate      

opt = Adam(lr=0.0)
lrate = LearningRateScheduler(step_decay)

train_gen = generate_images(train_idx, is_training=True, batch_size=32,AGE_CAT_NUM=7)
valid_gen = generate_images(val_idx, is_training=True, batch_size=32,AGE_CAT_NUM=7)
callbacks_list = [lrate]


model.compile(optimizer=opt, 
              loss={
                  'age_output': 'categorical_crossentropy', 
                  'gender_output': 'categorical_crossentropy'},
              metrics={
                  'age_output': 'accuracy', 
                  'gender_output': 'accuracy'})

history = model.fit_generator(train_gen, steps_per_epoch = len(train_idx)//32, epochs=22  , callbacks=callbacks_list,
                     validation_data=valid_gen, validation_steps=len(val_idx)//32)



#,class_weight=class_weights (add class weights where applicable)



In [ ]:
fig, ax = plt.subplots(2,2,figsize=(20,10))

ax[0,0].plot(history.history['val_age_output_acc'],label='age val acc')
ax[0,0].plot(history.history['val_gender_output_acc'],label='gen val acc')
ax[0,0].legend(loc='lower right',ncol=3)

ax[0,1].plot(history.history['age_output_acc'],label='age train acc')
ax[0,1].plot(history.history['gender_output_acc'],label='gen train acc')
ax[0,1].legend(loc='lower right',ncol=3)


ax[1,0].plot(history.history['val_age_output_loss'],label='age val loss')
ax[1,0].plot(history.history['val_gender_output_loss'],label='gen val loss')
ax[1,0].legend(loc='upper right',ncol=3)

ax[1,1].plot(history.history['age_output_loss'],label='age train loss')
ax[1,1].plot(history.history['gender_output_loss'],label='gen train loss')
ax[1,1].legend(loc='upper right',ncol=3)


# Model Taxonomy (Training stagnates after 25 epochs)

##### ITERATION 1: Default run (age = 25 categories : epochs : 22 )

loss: 0.3473 - age_output_loss: 0.2929 - gender_output_loss: 0.0295 - age_output_acc: 0.8930 - gender_output_acc: 0.9908 - val_loss: 0.9879 - val_age_output_loss: 0.6240 - val_gender_output_loss: 0.3400 - val_age_output_acc: 0.8065 - val_gender_output_acc: 0.9169


##### ITERATION 2: Encoded age with 7 classes (age range = 10 : epochs : 22)

val_age_output_acc: 0.70 - 

##### ITERATION 3: Encoded age with 7 classes (age range = 10 : epochs : 50)

Epoch 50/50 804s 2s/step - loss: 0.1529 - age_output_loss: 0.1406 - gender_output_loss: 0.0034 - age_output_acc: 0.9529 - gender_output_acc: 0.9990 - val_loss: 1.6928 - val_age_output_loss: 1.2100 - val_gender_output_loss: 0.4740 - val_age_output_acc: 0.7325 - val_gender_output_acc: 0.9171

##### ITERATION 4: Encoded age with 7 classes and class weights (age range = 10 : epochs : 50)

loss: 0.2539 - age_output_loss: 0.2343 - gender_output_loss: 0.0086 - age_output_acc: 0.9187 - gender_output_acc: 0.9976 - val_loss: 1.6371 - val_age_output_loss: 1.2031 - val_gender_output_loss: 0.4230 - val_age_output_acc: 0.7076 - val_gender_output_acc: 0.9169

# CODE FOR SAVING 

In [ ]:
ITERATION =  

In [ ]:
saveModelHistory(history,ITERATION,ADIENCE_PATHS['MODEL_HISTORY'])
model.save(ADIENCE_PATHS['MODEL_SAVED'].format(ITERATION)) 